모델 준비 : Kobert 모델을 준비한다.
Load the KO-BERT model:

In [13]:
from transformers import BertTokenizer, BertModel

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertModel.from_pretrained("monologg/kobert")

Prepare your Korean language data:

Collect a dataset containing Korean sentences or documents.
Preprocess the text (e.g., remove special characters, convert to lowercase) if necessary.

In [14]:
character_limit = 300
def read_text_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    sentences = []
    for line in lines:
        line = line.strip()  # Remove leading/trailing whitespace
        while len(line) > character_limit:
            sentences.append(line[:character_limit])  # Add the first 500 characters
            line = line[character_limit:]  # Remove the first 500 characters
        sentences.append(line)  # Add the remaining characters as a sentence

    return sentences


In [15]:
# todo

data = read_text_file("KCCq28_Korean_sentences_UTF8_v2.txt")

In [16]:
data = data[:2000]

Tokenize and encode the data:

Tokenize the Korean text using the KO-BERT tokenizer.
Convert the tokenized text into input features that the model can understand.

In [17]:
# Tokenize and encode the data
def encode_text(text):
    tokens = tokenizer.tokenize(text)
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    attention_mask = [1] * len(input_ids)

    # Pad or truncate the input to a fixed length
    max_length = 128
    padding_length = max_length - len(input_ids)
    input_ids = input_ids + ([0] * padding_length)
    attention_mask = attention_mask + ([0] * padding_length)

    return input_ids, attention_mask


In [18]:
# Encode the data
encoded_data = [encode_text(text) for text in data]
input_ids = [enc[0] for enc in encoded_data]
attention_masks = [enc[1] for enc in encoded_data]

Vectorize the data using KO-BERT:

Pass the encoded input through the KO-BERT model to obtain the hidden states, which represent the contextualized embeddings for each token.
Extract the representation for the [CLS] token as the sentence or document embedding.

In [19]:
import torch

# Convert the inputs to PyTorch tensors
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

# Set the device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = input_ids.to(device)
attention_masks = attention_masks.to(device)

# Obtain the embeddings
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)
    embeddings = outputs[0][:, 0, :].cpu().numpy()  # Extract [CLS] token embeddings


In [20]:
embeddings

array([[ 0.01500012,  0.06387398,  0.3232647 , ..., -0.21675886,
        -0.49001923, -0.37492698],
       [-0.13425061,  0.00895801,  0.46948475, ..., -0.04127207,
        -0.44758508, -0.2053517 ],
       [-0.24918352, -0.26243773,  0.36645257, ..., -0.33158505,
        -0.25104234, -0.13303171],
       ...,
       [-0.112602  ,  0.16751897,  0.42021683, ..., -0.23842525,
        -0.20740634, -0.24527523],
       [-0.08303856,  0.1579861 ,  0.39927542, ..., -0.2115599 ,
        -0.14765759, -0.05824763],
       [-0.04343085,  0.05151055,  0.8291116 , ..., -0.36878407,
         0.05955462,  0.01502977]], dtype=float32)

Build the FAISS index:

Create an index object with the desired configuration (e.g., index type, dimensionality).
Add the normalized embeddings to the index.

In [21]:
import faiss

# Build the index
index = faiss.IndexFlatIP(embeddings.shape[1])  # Index type for inner product (cosine similarity)
index.add(embeddings)


Perform similarity search using FAISS:

Encode the query text using KO-BERT and obtain the query embedding.
Normalize the query embedding.
Search for similar embeddings in the FAISS index.
Retrieve the indices and distances of the most similar embeddings.
Retrieve the corresponding sentences or documents.



In [22]:
# Example: Perform similarity search
query_text = "그러면서 우리가 추구하는 가치와 경영이념, 임직원들의 뜻과 마음을 담아 사회공헌의 주제를 정하고, "

In [28]:
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

query_input = encode_text(query_text)
query_input = [torch.tensor(tensor).to(device) for tensor in query_input]

query_embedding = model(query_input[0].unsqueeze(0), attention_mask=query_input[1].unsqueeze(0))[0][:, 0, :]
query_embedding = query_embedding.cpu().detach().numpy()

normalized_query_embedding = query_embedding / np.linalg.norm(query_embedding)

# Search for similar embeddings
k = 5  # Number of similar embeddings to retrieve
distances, indices = index.search(normalized_query_embedding, k)

# Retrieve similar sentences
similar_sentences = [data[i] for i in indices[0]]

In [29]:
similar_sentences

['그럼 순수한 어린이 관객들은 쉽게 외면하죠"그는 왜곡된 뮤지컬이 실패하면 다시 투자를 받기 어려워지고, 이를 만회하기 위해 저가의 작품이 만들어진다며 안타까워했다.',
 '로이코 관계자에 따르면 "805 마세라티 에디션은 단순히 마세라티의 이름을 빌린 것이 아니라 마세라티가 직접 우드를 선별할 정도로 많은 정성이 들어간 제품이다.',
 '그러면서 "이렇게 국가적 아젠다에 대해 고민하는 것은 기본적 애국심의 발로인데 어찌해서 국민들에게 하나마나 한 맹탕 개혁이다, 졸속이다, 비열한 거래다, 이런 말로 매도 당하는지 정말 기가 막힌 심정"이라고 강조했다.',
 '하성란씨의 \'카레 온 더 보더\'는 세상의 온갖 비린내를 가려주는 카레라는 음식에 기대 팍팍한 삶을 견뎌온 등장인물들을 통해 청년 실업과 노인 문제를 다룬 작품으로 "사회성과 언어성·문학성을 정교하게 결합한 수작"이라는 평을 들었다.',
 '그러면서 "청렴연수원을 국제적인 반부패 교육의 허브로 육성하고, \'공직자 부정청탁금지 및 이해충돌방지법\' 제정에도 노력해 청탁 관행 근절을 위한 인프라를 공고히 해야 할 것"이라고 덧붙였다.']